# Einführung in Wahrscheinlichkeit und Statistik
In diesem Notebook werden wir mit einigen der zuvor besprochenen Konzepte spielen. Viele Konzepte aus Wahrscheinlichkeit und Statistik sind gut vertreten in wichtigen Bibliotheken für die Datenverarbeitung in Python, wie `numpy` und `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Zufallsvariablen und Verteilungen
Beginnen wir damit, eine Stichprobe von 30 Werten aus einer Gleichverteilung von 0 bis 9 zu ziehen. Wir werden auch Mittelwert und Varianz berechnen.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Um visuell abzuschätzen, wie viele verschiedene Werte in der Stichprobe vorhanden sind, können wir das **Histogramm** zeichnen:


In [ ]:
plt.hist(sample)
plt.show()

## Analyse realer Daten

Mittelwert und Varianz sind beim Analysieren realer Daten sehr wichtig. Laden wir die Daten über Baseballspieler von [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights) herunter.


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Wir verwenden hier ein Paket namens [**Pandas**](https://pandas.pydata.org/) für die Datenanalyse. Später in diesem Kurs werden wir mehr über Pandas und die Arbeit mit Daten in Python sprechen.

Lassen Sie uns den Durchschnittswert für Alter, Größe und Gewicht berechnen:


In [ ]:
df[['Age','Height','Weight']].mean()

Kommen wir nun zur Größe und berechnen die Standardabweichung und Varianz:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Zusätzlich zum Mittelwert ist es sinnvoll, den Median und die Quartile zu betrachten. Diese können mithilfe eines **Boxplots** visualisiert werden:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Wir können auch Boxplots von Teilmengen unseres Datensatzes erstellen, zum Beispiel gruppiert nach Spielerrolle.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Hinweis**: Dieses Diagramm legt nahe, dass im Durchschnitt die Körpergrößen der First Basemen höher sind als die der Second Basemen. Später werden wir lernen, wie wir diese Hypothese formaler testen können und wie wir zeigen können, dass unsere Daten statistisch signifikant sind, um dies zu belegen.  

Alter, Größe und Gewicht sind alle stetige Zufallsvariablen. Was denkst du, wie ihre Verteilung aussieht? Eine gute Möglichkeit, das herauszufinden, ist es, ein Histogramm der Werte zu zeichnen: 


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normalverteilung

Lassen Sie uns eine künstliche Stichprobe von Gewichten erstellen, die einer Normalverteilung mit demselben Mittelwert und derselben Varianz wie unsere echten Daten folgt:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Da die meisten Werte im wirklichen Leben normalverteilt sind, sollten wir keinen gleichmäßigen Zufallszahlengenerator verwenden, um Beispieldaten zu erzeugen. So sieht es aus, wenn wir versuchen, Gewichte mit einer gleichmäßigen Verteilung (generiert mit `np.random.rand`) zu erzeugen:


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Konfidenzintervalle

Lassen Sie uns nun Konfidenzintervalle für das Gewicht und die Größe von Baseballspielern berechnen. Wir verwenden den Code [aus dieser Stackoverflow-Diskussion](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data):


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Hypothesentest

Lassen Sie uns verschiedene Positionen in unserem Baseball-Spielerdatensatz untersuchen:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Lassen Sie uns die Hypothese testen, dass Erste Basisspieler größer sind als Zweite Basisspieler. Die einfachste Methode dafür ist, die Konfidenzintervalle zu überprüfen:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Wir können sehen, dass sich die Intervalle nicht überschneiden.

Eine statistisch korrektere Methode, um die Hypothese zu überprüfen, ist die Verwendung eines **Student-t-Tests**:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

Die beiden von der Funktion `ttest_ind` zurückgegebenen Werte sind:  
* Der p-Wert kann als die Wahrscheinlichkeit betrachtet werden, dass zwei Verteilungen den gleichen Mittelwert haben. In unserem Fall ist er sehr niedrig, was darauf hinweist, dass es starke Hinweise darauf gibt, dass erste Basemen größer sind.  
* Der t-Wert ist der Zwischenschritt des normalisierten Mittelwertunterschieds, der im t-Test verwendet wird und mit einem Schwellwert für einen bestimmten Vertrauenswert verglichen wird.


## Simulation einer Normalverteilung mit dem Zentralen Grenzwertsatz

Der Pseudozufallsgenerator in Python ist so konzipiert, dass er uns eine gleichmäßige Verteilung liefert. Wenn wir einen Generator für die Normalverteilung erstellen möchten, können wir den zentralen Grenzwertsatz verwenden. Um einen normalverteilten Wert zu erhalten, berechnen wir einfach den Mittelwert einer gleichmäßig generierten Stichprobe.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korrelation und Evil Baseball Corp

Korrelation ermöglicht es uns, Beziehungen zwischen Datenfolgen zu finden. In unserem einfachen Beispiel tun wir so, als gäbe es eine böse Baseballfirma, die ihre Spieler entsprechend ihrer Körpergröße bezahlt – je größer der Spieler ist, desto mehr Geld erhält er/sie. Angenommen, es gibt ein Grundgehalt von 1000 $, und einen zusätzlichen Bonus von 0 bis 100 $, abhängig von der Körpergröße. Wir nehmen die echten Spieler der MLB und berechnen deren imaginäres Gehalt:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Berechnen wir nun die Kovarianz und Korrelation dieser Sequenzen. `np.cov` liefert uns eine sogenannte **Kovarianzmatrix**, die eine Erweiterung der Kovarianz auf mehrere Variablen darstellt. Das Element $M_{ij}$ der Kovarianzmatrix $M$ ist eine Kovarianz zwischen den Eingangsvariablen $X_i$ und $X_j$, und die Diagonalwerte $M_{ii}$ sind die Varianz von $X_{i}$. Ähnlich liefert `np.corrcoef` uns die **Korrelationsmatrix**.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Eine Korrelation von 1 bedeutet, dass eine starke **lineare Beziehung** zwischen zwei Variablen besteht. Wir können die lineare Beziehung visuell sehen, indem wir einen Wert gegen den anderen auftragen:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Mal sehen, was passiert, wenn die Beziehung nicht linear ist. Angenommen, unser Unternehmen hätte beschlossen, die offensichtliche lineare Abhängigkeit zwischen Größen und Gehältern zu verbergen und eine gewisse Nicht-Linearität in die Formel einzuführen, wie zum Beispiel `sin`:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In diesem Fall ist die Korrelation etwas geringer, aber immer noch ziemlich hoch. Nun, um die Beziehung noch weniger offensichtlich zu machen, könnten wir etwas zusätzliche Zufälligkeit hinzufügen, indem wir eine Zufallsvariable zum Gehalt addieren. Mal sehen, was passiert:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Können Sie erraten, warum die Punkte sich zu solchen vertikalen Linien anordnen?

Wir haben die Korrelation zwischen einem künstlich entwickelten Konzept wie dem Gehalt und der beobachteten Variable *Größe* untersucht. Schauen wir uns nun an, ob die beiden beobachteten Variablen, wie Größe und Gewicht, ebenfalls korrelieren:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Leider haben wir keine Ergebnisse erhalten – nur einige seltsame `nan`-Werte. Dies liegt daran, dass einige Werte in unserer Serie undefiniert sind, dargestellt als `nan`, was dazu führt, dass auch das Ergebnis der Operation undefiniert ist. Wenn wir uns die Matrix ansehen, erkennen wir, dass `Weight` die problematische Spalte ist, da die Selbstkorrelation zwischen den `Height`-Werten berechnet wurde.

> Dieses Beispiel zeigt die Bedeutung von **Datenvorbereitung** und **Datenbereinigung**. Ohne richtige Daten können wir nichts berechnen.

Lassen Sie uns die Methode `fillna` verwenden, um die fehlenden Werte zu füllen, und die Korrelation berechnen: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Es gibt tatsächlich eine Korrelation, aber keine so starke wie in unserem künstlichen Beispiel. Wenn wir uns tatsächlich das Streudiagramm eines Werts gegen den anderen ansehen, wäre die Beziehung viel weniger offensichtlich:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Fazit

In diesem Notebook haben wir gelernt, wie man grundlegende Operationen an Daten durchführt, um statistische Funktionen zu berechnen. Wir wissen jetzt, wie man ein solides Arsenal an Mathematik und Statistik einsetzt, um Hypothesen zu überprüfen, und wie man Konfidenzintervalle für beliebige Variablen anhand einer Datenstichprobe berechnet. 


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir um Genauigkeit bemüht sind, können automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten. Das Originaldokument in seiner Ausgangssprache sollte als maßgebliche Quelle angesehen werden. Für wichtige Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die aus der Nutzung dieser Übersetzung entstehen.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
